In [52]:
# Useful libs
from sklearn.datasets import fetch_20newsgroups
import pickle
import pandas as pd
import numpy as np
import re

# STTM lib from Github
from gsdmm.gsdmm import MovieGroupProcess


# Load the 20NewsGroups dataset from sklearn
cats = ['talk.politics.mideast', 'comp.windows.x', 'sci.space']
newsgroups_train = fetch_20newsgroups(subset='train', categories=cats)

In [53]:
import spacy

nlp = spacy.load("en_core_web_sm")
doc = nlp(newsgroups_train.data[5])

stopwords -> strip ->  acentos -> correcao (pal. sem acento) -> entidades -> minusculo -> 

ok ok ok correcao entidades minusculo

Realizacao de correção https://norvig.com/spell-correct.html


In [255]:
tokenization(pd.Series(['Eu amo batatas','Porem adogo X-Pastrami que é 20.00 no www.google.com']))

[['Eu', 'amo', 'batatas'],
 ['Porem', 'adogo', 'X-Pastrami', '20.00', 'www.google.com']]

In [276]:
import nltk
from sklearn.feature_extraction.text import strip_accents_unicode
from nltk.tokenize import SpaceTokenizer


def tokenization(text_data ,spwrds = nltk.corpus.stopwords.words('portuguese')):
    
    '''Input : Text Data precisa ser uma série para recebimento do corpus
    Output : Uma lista de listas com palavras 
    
    Esta função possui um argumento para remocao de stopwords'''
    
    #Tokenizacao
    spacetok = SpaceTokenizer()
    text_data = [spacetok.tokenize(phrases) for phrases in text_data]
    
    
    #stopwords
    if(spwrds != None):
        tmp_text_data = list()
        tmp_words = list()

        for phrases in text_data:
            for word in phrases:
                if(word not in spwrds):
                    tmp_words.append(word)
                else:
                    pass
            tmp_text_data.append(tmp_words)
            tmp_words = list()

        text_data = tmp_text_data
    else:
        pass
    
    return text_data

def entities_subs(text_data):
    
    #regex_numbers 
    text_data = [re.sub(r'\d+','<NUM>', phrases) for phrases in text_data]
    
    #regex_links
    text_data = [re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+',
                        '<LINK>', phrases) for phrases in text_data]
    text_data = [re.sub('www.(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+',
                        '<LINK>', phrases) for phrases in text_data]

    #NERs
    
    return text_data

def text_correction(text_data):
    pass
    return text_data
    
def rule_preprocessing(text_data, stop_words = nltk.corpus.stopwords.words('portuguese')):
    
    '''Inicialmente trabalharemos com os documentos pertencentes ao corpus, depois daremos um olhar mais 
    aprofundado nas palavras'''
    
    #strip para ter certeza de espacos
    text_data = [phrases.strip() for phrases in text_data]
    text_data = [re.sub(' +',' ', phrases) for phrases in text_data]
    
    #coloca todas as palavras em minúsculo
    text_data = [phrases.lower() for phrases in text_data]
    
    #Realizacao de substituicao de entidades
    text_data = entities_subs(text_data)
    
    #correcao palavras
    text_data = text_correction(text_data)
    
    #retira acentos
    text_data = [strip_accents_unicode(phrases) for phrases in text_data]

    text_data = tokenization(text_data ,spwrds = stop_words)
    
    #text_data = [rgxtoken.tokenize(phrases) for phrases in text_data]
    
    return text_data

In [277]:
test_list = ['   Eu          tenho um site muito loco www.zzz.com.br https://github.com/Franck-Dernoncourt/NeuroNER entendeu?',
'este é mais um teste que vale um milhão de reais R$ 10000']

In [278]:
rule_preprocessing(test_list)

[['site', 'loco', '<LINK>', '<LINK>', 'entendeu?'],
 ['teste', 'vale', 'milhao', 'reais', 'r$', '<NUM>']]

In [265]:
spacetok = SpaceTokenizer()
spacetok.tokenize(rule_preprocessing(test_list)[0])

AttributeError: 'list' object has no attribute 'split'

In [146]:
rule_preprocessing(test_list)

['eu tenho um site muito loco <LINK> <LINK> entendeu?',
 'este e mais um teste que vale um milhao de reais r$ <NUM>']

In [46]:
news_df = list()
for token in doc:
    news_df.append((token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop,token.has_vector, token.vector_norm, token.is_oov))

pd.DataFrame(news_df,columns = ['text','lemma','pos','tag','dep','shape','is_alpha','is_stop',
                               'has_vector','vector_norm','is_oov'])

,text,lemma,pos,tag,dep,shape,is_alpha,is_stop,has_vector,vector_norm,is_oov
0,From,from,ADP,IN,ROOT,Xxxx,True,True,True,22.316801,True
1,:,:,PUNCT,:,punct,:,False,False,True,19.780769,True
2,arf@genesis,arf@genesis,PROPN,NNP,pobj,xxx@xxxx,False,False,True,21.910822,True
3,.,.,PUNCT,.,punct,.,False,False,True,20.598114,True
4,MCS.COM,MCS.COM,PROPN,NNP,ROOT,XXX.XXX,False,False,True,20.771774,True
5,(,(,PUNCT,-LRB-,punct,(,False,False,True,22.290766,True
6,Jack,Jack,PROPN,NNP,compound,Xxxx,True,False,True,23.171431,True
7,Schmidling,Schmidling,PROPN,NNP,ROOT,Xxxxx,True,False,True,22.160078,True
8,),),PUNCT,-RRB-,punct,),False,False,True,23.919390,True
9,\n,\n,SPACE,_SP,,\n,False,False,True,22.096819,True


In [15]:
newsgroups_train.target

array([1, 0, 2, ..., 2, 1, 0])

In [28]:
print(newsgroups_train.data[7])

From: tracy@cs.ucf.edu (Tracy Rene Tolley)
Subject: HELP: xdm & Solaris2.1
Summary: help with xdm & Solaris2.1
Keywords: Solaris2.1 xdm
Organization: University of Central Florida
Distribution: comp
Lines: 7

I recently read here that Sun has a patch for xdm on
Solaris2.1.  I was wondering if anyone could give me the 
patch number. 

Thanks in advance,
Tracy Tolley
University of Central Florida - Technical Support



In [6]:
from nltk.tokenize import RegexpTokenizer
from nltk.stem import RSLPStemmer

regxtok = RegexpTokenizer('/w+')
rslps = RSLPStemmer()

In [13]:
pd.DataFrame(newsgroups_train)

ValueError: arrays must all be same length

In [ ]:
regxtok.tokenize()


In [4]:
# Preprocessing and tokenization
tokenized_data = tokenize(df, form_reduction='stemming', predict=False)

NameError: name 'tokenize' is not defined